In [24]:
!pip install -U deep-translator
!pip install edge-tts
!pip install PyPDF2
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=552e3c6be527549eb3194e0d463c3e25d92bf7e08f843f48c92eb791ced92f4e
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [34]:
# Código base para extrair o texto de um PDF qualquer

import os
import logging
from PyPDF2 import PdfReader
import edge_tts
from edge_tts import VoicesManager
from deep_translator import GoogleTranslator
from deep_translator import single_detection
from langdetect import detect


def extrair_livro_pdf(file_full_path):
    '''
    Extrai o conteúdo de um livro pdf a partir do caminho base,
    verificando se o conteúdo já foi salvo em um arquivo JSON.
    Args:
        file_full_path (str): caminho base do livro.
    Returns:
        todotexto: variável string com o conteudo filtrado.
    '''

    # Abrindo um arquivo PDF existente
    with open(file_full_path, "rb") as input_pdf:
       # Criando um objeto PdfFileReader
       pdf_reader = PdfReader(input_pdf)

       # Obtendo o número de páginas do arquivo PDF
       num_pages = len(pdf_reader.pages)

       # Lendo o texto de somente 2 páginas
       workstr = ''
       todotexto = ''
       for page_number in range(2):
         page = pdf_reader.pages[page_number]
         workstr = page.extract_text()
         workstr = workstr.replace("\n", "")
         todotexto = todotexto + workstr

    return todotexto


print('Indique o livro conforme a lista abaixo')

caminho = r'/content/sample_data/'
lista = []
# Iterate directory
i = 0
for file in os.listdir(caminho):
    if file.endswith('.pdf'):
        lista.append(file)
        i = i+1
        print('[',i,'] ',file)
if not lista :
    print('\nNenhum arquivo pdf encontrado. Saindo...')
    os._exit(0)
escolha = input()
livroEscolhido = lista[int(escolha)-1]
print (livroEscolhido)

print('\nIndique o idioma desejado:')
print('[0] - Francês')
print('[1] - Inglês')
print('[2] - Português - default')
idioma = input()
if idioma == '0' : codIdioma = 'fr'
elif idioma == '1' : codIdioma = 'en'
elif idioma == '2' : codIdioma = 'pt'
else : codIdioma = 'pt-BR'

print('\nIndique o gênero da voz desejada')
print('[0] - Masculino')
print('[1] - Feminino')
print('[2] - Qualquer um - default')
gVoz = input()
if gVoz == '0' : genero = 'Male'
elif gVoz == '1' : genero = 'Female'
else : genero = 'Qualquer um'

print('\nOpções disponíveis para a combinação de idioma e voz: ')
voices = await VoicesManager.create()
if genero == 'Qualquer um' :
  voice = voices.find(Language=codIdioma)
else : voice = voices.find(Gender=genero, Language=codIdioma)
i = 0
listaNomesAmigaveis = []
for v in voice:
  print('[',i,'] ',v["FriendlyName"])
  listaNomesAmigaveis.append(v["FriendlyName"])
  i = i+1
if not listaNomesAmigaveis :
  print('\nVoz não disponível. Saindo')
  os._exit(0)
idvozEscolhida = input()
voice = voices.find(FriendlyName=listaNomesAmigaveis[int(idvozEscolhida)])
idVoice = voice[0]["ShortName"]

file_full_path='/content/sample_data/'+livroEscolhido
sohtexto = extrair_livro_pdf(file_full_path)

lang = detect(sohtexto)
print(lang)
if lang != codIdioma :
   tradutor = GoogleTranslator(source='auto', target=codIdioma)
   texto = tradutor.translate(sohtexto)
else : texto = sohtexto

communicate = edge_tts.Communicate(texto, idVoice)
arquivo = livroEscolhido + ".mp3"
communicate.save_sync(arquivo)
print('\nÁudio gerado')


Indique o livro conforme a lista abaixo
[ 1 ]  GordonFlavio-GlobalismoComunismo.pdf
[ 2 ]  Confissoes.pdf
[ 3 ]  AnimalFarm.pdf
2
Confissoes.pdf

Indique o idioma desejado:
[0] - Francês
[1] - Inglês
[2] - Português - default
1

Indique o gênero da voz desejada
[0] - Masculino
[1] - Feminino
[2] - Qualquer um - default
1

Opções disponíveis para a combinação de idioma e voz: 
[ 0 ]  Microsoft Natasha Online (Natural) - English (Australia)
[ 1 ]  Microsoft Clara Online (Natural) - English (Canada)
[ 2 ]  Microsoft Yan Online (Natural) - English (Hong Kong SAR)
[ 3 ]  Microsoft Neerja Online (Natural) - English (India) (Preview)
[ 4 ]  Microsoft Neerja Online (Natural) - English (India)
[ 5 ]  Microsoft Emily Online (Natural) - English (Ireland)
[ 6 ]  Microsoft Asilia Online (Natural) - English (Kenya)
[ 7 ]  Microsoft Molly Online (Natural) - English (New Zealand)
[ 8 ]  Microsoft Ezinne Online (Natural) - English (Nigeria)
[ 9 ]  Microsoft Rosa Online (Natural) - English (Philippines)